In [2]:
import sqlalchemy

sqlalchemy.__version__

'1.3.18'

In [3]:
from sqlalchemy import Table, Column, Integer, String, MetaData

metadata = MetaData()

users = Table(
    'users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('fullname', String),
)

phones = Table(
    'phones', metadata,
    Column('id', Integer, primary_key=True),
    Column('user_id', Integer),
    Column('number', String),
)

In [4]:
from sqlalchemy import select

print(select([users.c.name]))

SELECT users.name 
FROM users


In [5]:
u_query = select([users.c.id]).where(users.c.name == 'Dima')
u_cte = select([users.c.id]).where(users.c.name == 'Dima').cte(name='eva00')
print(u_query)

SELECT users.id 
FROM users 
WHERE users.name = :name_1


In [6]:
q2 = select([(phones.c.number + u_cte.c.id).label('eva11')]).where(phones.c.user_id == u_cte.c.id)

print(q2)

WITH eva00 AS 
(SELECT users.id AS id 
FROM users 
WHERE users.name = :name_1)
 SELECT phones.number + eva00.id AS eva11 
FROM phones, eva00 
WHERE phones.user_id = eva00.id


In [7]:
u_cte_1 = select([users.c.id]).where(users.c.name == 'Dima').cte(name='eva00')

u_cte == u_cte_1

False

In [8]:
q3 = select([(phones.c.number + u_cte_1.c.id).label('eva11')]).where(phones.c.user_id == u_cte_1.c.id)
print(q3)

WITH eva00 AS 
(SELECT users.id AS id 
FROM users 
WHERE users.name = :name_1)
 SELECT phones.number + eva00.id AS eva11 
FROM phones, eva00 
WHERE phones.user_id = eva00.id


In [9]:
from sqlalchemy import literal

u_cte_1 = select([literal(1).label('col_1')]).cte(name='eva00')

q = select([users.c.id + u_cte_1.c.col_1])

print(q)

WITH eva00 AS 
(SELECT :param_1 AS col_1)
 SELECT users.id + eva00.col_1 AS anon_1 
FROM users, eva00


In [10]:
u_cte_2 = select([literal(1).label('col_1')]).cte(name='eva00')

q1 = select([users.c.id + u_cte_1.c.col_1]).where(users.c.id == u_cte_2.c.col_1)

print(q1)

CompileError: Multiple, unrelated CTEs found with the same name: 'eva00'

In [32]:
u_id_cte = select([users.c.id]).cte(name='us_id')

u_id_cte_1 = select([users.c.id]).cte(name='us_id')
up_id_cte = select([phones.c.user_id]).where(phones.c.user_id == u_id_cte_1.c.id).cte(name='up_id')

result_query = select([phones.c.number]).where((phones.c.user_id == up_id_cte.c.user_id) & (phones.c.user_id == u_id_cte.c.id))

print(result_query)

CompileError: Multiple, unrelated CTEs found with the same name: 'us_id'

In [30]:
from sqlalchemy import column
from sqlalchemy.sql.base import ImmutableColumnCollection

dir(select([column('id')]).select_from(users).cte().columns)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__self_class__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__thisclass__',
 'id']